# WGTDA Tutorial

This tutorial will guide you through using WGTDA to identify biomarkers for your gene expression data. 

## Steps of Tutorial
1. Importing WGTDA modules.
2. Load Gene expression data and apply preprocessing steps.

    A. Load gene expression data

    B. Filtering gene expression data from txt file.

    C. Creating gene dictionary for simplicial complex.

3. Compute Gene-Gene relationship matrix as input to simplicial complex

    A. Compute Distance Correlation matrix

    B. Compute Signed Topological Overlapping Measures matrix

4. Construct the Vietoris-Rips Complex on the gene-gene relationship matrix.
5. Filter non-informative topological features.

    A. Remove features that persist to infinite.
    
    B. Remove features that do not persist across user-defined thereshold (noise).

6. Identify Biomarkers using in-vitro and in vivo  methods such as Survival Analysis and  


## Step 1: Import WGTDA Modules

First, we need to import the necessary libraries from WGTDA. This includes persistent homology and simplicial complex functions, correlation matrix functions that include distance correlation and weighted topological overlapping measures and filters for the topological interactions.  

In [1]:
import pandas as pd
import numpy as np
import ast
from wgtda.correlation import compute_distance_correlation_matrix, compute_wto_matrix
from wgtda.filters import extract_top_n_persistent_holes, remove_infinite_holes
from wgtda import (
    construct_vr_complex_rna_matrix,
    interactions_dataframe,
    filter_genes,
    convert_gene_exp_to_array_and_dict,
    flatten_gene_list,
)
from wgtda.networks import create_network_graph

## Step 2: Load Gene expression data and apply preprocessing steps.

    A. Load gene expression data.
    B. Filtering gene expression data from txt file.
    C. Creating gene dictionary for simplicial complex.

In [2]:
# Load the gene expression data from a .pkl file
gene_expression_df = pd.read_pickle("../data/TCGA/BRCA.pkl")


# Filter the gene expression data using the gene list from cancer_genes.txt
gene_expression_filtered = filter_genes(
    gene_expression_df, "../data/preselection/cancer_genes.txt"
)

gene_expression_filtered_array, gene_dict = convert_gene_exp_to_array_and_dict(
    gene_expression_filtered
)

print("First 5 rows of gene expression data: ")
print(gene_expression_filtered.head())
print("Gene Dictionary: ", gene_dict)

Number of genes filtered:  51
First 5 rows of gene expression data: 
gene_name    DVL2   CREBBP    E2F2   RALBP1   CTNNA1   LAMC3  CTNNA2   RASSF1  \
P1         4.0276   7.1788  2.1384  11.1616  65.2214  0.3504  0.0246   6.1498   
P2         3.3894   9.2800  0.7042  14.3794  62.2928  0.3500  0.0053   5.3676   
P3         5.3978  11.3675  1.3278  10.9282  77.2190  0.4100  0.0051   6.9546   
P4         4.8691  10.3431  0.8642  15.3211  80.3090  1.3886  0.0000   5.2736   
P5         3.1467   9.6298  3.8233   8.0495  61.6703  0.0417  0.0038  13.2769   

gene_name  COL4A4     CRKL  ...    CSF1R     DCC   COL4A1   DAPK1  COL4A6  \
P1         0.1629  19.3946  ...  23.0582  0.1092  58.4540  2.7505  0.3275   
P2         0.1032  21.6408  ...   3.8766  0.0228  46.3063  0.9933  0.1116   
P3         0.1720  33.0858  ...   5.7958  0.0029  45.2401  1.8868  0.4610   
P4         0.2019  11.9940  ...  10.8742  0.0133  66.1155  4.3019  0.9681   
P5         0.3823  15.8862  ...  10.2689  0.0055  19.3364  

## Step 3: Compute gene-gene relationship matrix (distance correlation matrix or signed topological overlapping matrix)

### Distance Correlation

A gene-gene pairwise distance correlation matrix was computed from the gene expression data, capturing the statistical
relationships between each pair of genes and providing a comprehensive view of their co-expression patterns.

$ kME(i,M) = \dfrac {cov(X_i ,X_m)} {\sqrt{var(X_i)\cdot var(X_m)}} $

Where $ cov(X_i, X_M)$ is the covariance between the expression profile of gene $i$ (denoted as $X_i$) and the
module eigengene $X_M$, $var(X_i)$ is the variance of the expression profile of gene $i$, and $var(X_M)$ is the
variance of the module eigengene. A higher $kME$ value indicates a stronger correlation between the gene
and the module eigengene, signifying the genes role as a hub gene within the network. Hub genes are pivotal in gene co-expression networks, significantly influencing the network structure. They serve as crucial
biomarkers, representing collective expression dynamics. 

Here is how to use the Distance Correlation function:

```python
from wgtda.correlation import compute_distance_correlation_matrix
distance_corr_matrix = compute_distance_correlation_matrix(gene_exp_arr=gene_expression_filtered)
```

In [3]:
# Compute the Distance Correlation matrix
distance_corr_matrix = compute_distance_correlation_matrix(
    gene_exp_arr=gene_expression_filtered_array
)
print("Distance Correlation Matrix:\n", distance_corr_matrix)
np.savetxt(
    "relationship_matrices/brca_distance_correlation_matrix.csv",
    distance_corr_matrix,
    delimiter=",",
)

Distance Correlation Matrix:
 [[0.         0.78181019 0.83749928 ... 0.66437719 0.78486111 0.83344743]
 [0.78181019 0.         0.75006362 ... 0.48324453 0.71529892 0.69942692]
 [0.83749928 0.75006362 0.         ... 0.85965003 0.86740336 0.74244932]
 ...
 [0.66437719 0.48324453 0.85965003 ... 0.         0.71494494 0.83484625]
 [0.78486111 0.71529892 0.86740336 ... 0.71494494 0.         0.85443577]
 [0.83344743 0.69942692 0.74244932 ... 0.83484625 0.85443577 0.        ]]


### Signed Topological Overlaping Measures (wTO)

Signed Weighted Topological Overlap Measures (wTO) combine information from both the correlation and the shared network neighborhood of gene pairs to provide a comprehensive view of the gene network structure. This method captures both direct and indirect interactions among genes, making it a robust metric for network analysis. 

$ wTO(i,j) = \dfrac{W_{ij} + \sum_k\neq _i,_j W_{ik} \cdot W_{kj}}{min(\sum_k W_{ik}, \sum_k W_{jk}) + 1 - W_{ij}} $ 


Here is how to use the Signed Topological Overlapping Measure function:

```python
from wgtda.correlation import compute_wto
wto_matrix = compute_wto(gene_exp_arr=gene_expression_data)
```

In [4]:
# Compute the Signed Weighted Topological Overlap (wTO) matrix
wto_matrix = compute_wto_matrix(gene_exp_arr=gene_expression_filtered_array)
print("Signed Weighted Topological Overlap Matrix:\n", wto_matrix)
np.savetxt("relationship_matrices/brca_wto_matrix.csv", wto_matrix, delimiter=",")

Signed Weighted Topological Overlap Matrix:
 [[0.         0.81705039 0.81497841 ... 0.81674432 0.82147021 0.82096926]
 [0.81705039 0.         0.75378727 ... 0.78384256 0.82335606 0.82125272]
 [0.81497841 0.75378727 0.         ... 0.76575251 0.81938775 0.82433349]
 ...
 [0.81674432 0.78384256 0.76575251 ... 0.         0.82352169 0.82087678]
 [0.82147021 0.82335606 0.81938775 ... 0.82352169 0.         0.82063723]
 [0.82096926 0.82125272 0.82433349 ... 0.82087678 0.82063723 0.        ]]


## Step 4: Construct the Vietoris-Rips Complex on the gene-gene relationship matrix.

A simplicial complex is a construct encompassing an array of interconnected elements, ranging from
points to line segments, triangles, and their $n$-counterparts. Here, we utilize a Vietoris-Rips (VR) complex
which is a simplicial complex where its $k$-simplices are defined by subsets of $(k + 1)$ points within a set $X$,
each having a diameter that does not exceed a specified ϵ threshold.

The identification of topological features using TDA is centered around persistent homology, a mathematical tool that employs persistence-based filtration. In persistent homology, a filtration process is applied
to simplicial complexes using growing balls around each data point.  As these balls expand, the intersection
of growing balls between adjacent data points results in the formation of edges, connecting the data points
(Figure 1). This process continues, creating higher-dimensional simplices such as triangles and tetrahedra
as the filtration progresses.

Given a set of genes $X$ and a distance threshold $\epsilon$, the Vietoris-Rips complex is a simplicial complex where:

$ x_i $ is a vertex for each gene $x_i $  in $ X $.

$ x_i, x_j$ is an edge if $d(x_i, x_j) \leq \epsilon $.

$\{x_{i_0}, x_{i_1}, \ldots, x_{i_k}\}$ is a $k$-simplex if $d(x_{i_j}, x_{i_l}) \leq \epsilon$ for all $0 \leq j, l \leq k$.

1. Compute the distance matrix for the set of genes.
2. Add vertices for each genes.
3. Add edges for pairs of genes within the specified distance threshold.
4. Add higher-dimensional simplices for sets of genes that are all pairwise within the specified distance threshold.

The distance matrix $\mathbf{D}$ for a set of genes $ X = \{x_1, x_2, \ldots, x_n\}$ is given by:

$\mathbf{D}_{ij} = d(x_i, x_j) $

where $d(x_i, x_j)$ is either the distance correlation or the signed topological overlapping between genes $x_i$ and $x_j$.

```python
persistence, ripscomplex = construct_vr_complex_rna_matrix(relationship_matrix)

#interactions dataframe organises the topological features in a dataframe for post-analysis
interactions = interactions_dataframe(persistence, rips_complex, gene_dict)
```


In [5]:
persistence, rips_complex = construct_vr_complex_rna_matrix(distance_corr_matrix)
interactions = interactions_dataframe(persistence, rips_complex, gene_dict)

## Step 5: Filter non-informative topological features 

We filter topological features with the aim to mitigate noise and highlighting more reliable topological patterns. This is done by two functions namely, ```remove_infinite_holes``` and ```extract_persistent_holes```. These functions take in the interactions dataframe and filter the topological features the user would like to focus on.


```python
from wgtda.filters import extract_persistent_holes, remove_infinite_holes


# Remove the infinite holes first
interactions_df = remove_infinite_holes(interactions_df)

# Filter topological features to a user-defined n value (0<=100) 100 keeps all the topological featues.
interactions_df = extract_top_n_persistent_holes(interactions_df,n=10)
```



In [6]:
# Remove the infinite holes first
interactions = remove_infinite_holes(interactions)

# Filter topological features to a user-defined n value (0<=100) 100 keeps all the topological featues.
# In this example the user keeps the top 10% of persistent holes in the simplicial complex.
interactions = extract_top_n_persistent_holes(interactions, n=10)

In [7]:
print(interactions)

    interaction_id betti_number     birth     death  lifespan  \
47              47            0  0.000000  1.000000  1.000000   
25              25            0  0.000000  0.744217  0.744217   
14              14            0  0.000000  0.742740  0.742740   
40              40            0  0.000000  0.737072  0.737072   
45              45            0  0.000000  0.734656  0.734656   
56              56            1  0.624772  0.731421  0.106648   
55              55            1  0.621240  0.710859  0.089619   
62              62            1  0.691618  0.774089  0.082472   
86              86            2  0.734214  0.788065  0.053850   
93              93            2  0.755899  0.802754  0.046855   
107            107            2  0.794419  0.831165  0.036747   
94              94            2  0.757163  0.792148  0.034986   

                                              vertices  \
47                                                [47]   
25                                    

## Step 6: Identify biomarkers from the topological interactions.

Weighted Gene TDA (WGTDA) was implemented to identify novel biomarkers in gene expression data. WGTDA provides a topological perspective on the organization of gene expression networks, further deciphering the complexity embedded in sequence-based gene expression data. Survival analyses and topological network graphs on the identified gene signatures of both frameworks can be conducted to validate and establish the clinical relevance and utility of WGTDA and to identify biomarkers that are important for disease progression.

### Survival Analysis 
Survival analysis plays an important role in biomedical research by validating gene sets through in silico methods, evaluating biomarkers crucial for predicting patient outcomes, and determining the relationship between gene expression levels and survival probabilities.

![Surivial-Analysis](../docs/imgs/b1_luad_3.png)

### Network Graphs 
Network graphs reveal how genes interact within biological systems, highlighting connectivity patterns that underpin biological processes. Network graphs provide intuitive visualisations of spatial relationships between genes, offering insights into physical proximity which relate to functional associations to unravel biological complexity.

![Betti-1](../docs/imgs/Betti_1.png)
![Betti-2](../docs/imgs/Betti_2.png)
